<a href="https://colab.research.google.com/github/bbortey9/Big_Data-EDA-Regularization-/blob/main/Fraud_Detection_using_Kaggle_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from tensorflow import keras
from tensorflow.keras import layers
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,cross_val_score
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import RFE
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, roc_curve,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from statsmodels.stats.outliers_influence import variance_inflation_factor
import joblib
from sklearn import metrics
import statsmodels.api as sm

from sklearn.preprocessing import StandardScaler
import re  # Import the 're' module for regular expressions
sns.set()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Reading the data
Data=pd.read_csv('/content/drive/MyDrive/Fraud/creditcard.csv')

In [ ]:
Data

In [ ]:
Data.info()

In [ ]:
#Data = Data.drop(["Amount", "Time"], axis=1)

In [ ]:
Data

In [ ]:
#Check and drop deplicate columns
print('Before dedup:', Data.shape)
Data_cln=Data.loc[:, ~Data.columns.duplicated()]
print('After dedup:', Data_cln.shape)
duplicateCols=Data.loc[:, Data.columns.duplicated()]
if (duplicateCols.shape[1] !=0):
    print('Number of duplicated columns dropped:', duplicatedCols.shape[1])
    print("Dupliate columns except first occurrences:")
    print(list(duplicateCols.columns))

In [ ]:
#Checking Missing and Zero rate
def zero_missing_unique(df):
    var_miss_rate=df.isnull().sum(axis=0)/df.shape[0]
    var_miss_rate=var_miss_rate.to_frame('missing_rate')

    var_zero_rate=(df==0).astype(int).sum(axis=0)/df.shape[0]
    var_zero_rate=var_zero_rate.to_frame('zero_rate')
    var_unique=df.nunique().to_frame('count_unique')
    var_type=df.dtypes.to_frame('data_type')
    data_stat=pd.concat([var_miss_rate, var_zero_rate,var_unique, var_type], axis=1)
    return data_stat
stat_zero_missing_unique=zero_missing_unique(Data)# just change the data name here to your own data
stat_zero_missing_unique

In [ ]:
Data.describe().T

Apply Scalar Transformation to the Time and Amount Variables

In [ ]:
# Define the columns you want to standardize
columns_to_standardize = ['Time', 'Amount']

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scalar transformation to the selected columns
Data[columns_to_standardize] = scaler.fit_transform(Data[columns_to_standardize])


In [ ]:
#Creating dependent and independent indices
X_data=Data.drop(columns=['Class'])
Y_data=Data['Class']

In [ ]:
#Getting the correlation between the independent Variables
correlation_matrix=X_data.corr()
# Assuming df is your DataFrame with independent variables
correlation_matrix =X_data.corr()

# Create a mask to hide the upper triangle (to avoid duplication)
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Define a custom colormap
cmap = sns.color_palette("coolwarm", as_cmap=True)

# Set the threshold for the color scale
threshold = 0.95

# Create a heatmap with the custom colormap and threshold
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_matrix, annot=True, cmap=cmap, mask=mask, vmin=-threshold, vmax=threshold)

plt.show()

In [ ]:
threshold = 0.95
correlated_pairs = set()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            colname_i = correlation_matrix.columns[i]
            colname_j = correlation_matrix.columns[j]
            correlated_pairs.add((colname_i, colname_j))

In [ ]:
for var1, var2 in correlated_pairs:
    # Decide which variable to keep and remove the other
    # For example, removing var2:
    if var2 in X_data.columns:
        X_data.drop(var2, axis=1, inplace=True)

In [ ]:
# Split the data into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample

# Apply SMOTE to the target variable only
smote = SMOTE(sampling_strategy='auto',random_state=42)
X_resampled,y_resampled = smote.fit_resample(X_train,y_train)

In [ ]:

from collections import Counter
# Resampled class distribution
print("Class distribution after SMOTE (Target variable only):", Counter(y_train))

In [ ]:
# Sample DataFrame with the resampled target variable y_resampled
Y_resampled= pd.DataFrame({
    'y_resampled': y_resampled  # Replace with your actual resampled target variable
})


In [ ]:
# Count the occurrences of each class
class_counts =Y_resampled['y_resampled'].value_counts()

# Create a bar plot
plt.figure(figsize=(8, 6))
class_counts.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Class Distribution (After SMOTE)')
plt.show()

In [ ]:
# Create a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=10)

# Fit the Random Forest model on the training data
clf.fit(X_resampled, y_resampled)

# Get feature importances
importance_scores = clf.feature_importances_

# Get feature names
feature_names = X_resampled.columns

In [ ]:
# Create a DataFrame to store feature names and importance scores
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance_scores})

# Sort the DataFrame by importance scores in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Select the top 20 features based on importance scores
top_20_features_Random= feature_importance_df.head(20)

In [ ]:
# Plot feature importance scores
plt.figure(figsize=(12, 6))
plt.barh(top_20_features_Random['Feature'], top_20_features_Random['Importance'], color='skyblue')
plt.xlabel('Importance Score')
plt.title('Top 20 Features with Highest Importance Scores')
plt.gca().invert_yaxis()  # Invert the y-axis for better readability
plt.show()

In [ ]:
top_20_features_Random

In [ ]:
# Extract the names of the top features from the DataFrame
final_feature_names = top_20_features_Random['Feature'].tolist()
# Create a new DataFrame with only the selected top features
X_train = X_train[final_feature_names]
X_test=X_test[final_feature_names]

In [ ]:
# Convert X_train and y_train to NumPy arrays
X_train_np= np.array(X_train)
y_train_np= np.array(y_train)

In [ ]:
#Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(kernel='rbf', probability=True)))
#models.append(('SVM', SVC(kernel='rbf')))
models.append(('RF', RandomForestClassifier()))
models.append(('XGB', XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic', nthread=1, eval_metric='rmse')))
models.append(('LightGBM', lgb.LGBMClassifier()))

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, roc_curve, auc, confusion_matrix, average_precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer  # Import make_scorer


In [ ]:
# Initialize dictionaries to store results
results_train_cv = {'Model': [], 'KS_Train_CV': [], 'AUC_Train_CV': [], 'F1-Score_Train_CV': [], 'Recall_Train_CV': [], 'PRAUC_Train_CV': [],'Precision_Train_CV': []}

In [ ]:
# Define a custom scoring function for KS
def custom_ks_scorer(y_true, y_pred_prob):
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    return max(tpr - fpr)


In [ ]:

# Define a custom scoring function for KS
def custom_ks_scorer(y_true, y_pred_prob):
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    return max(tpr - fpr)



In [ ]:
# Define K-Fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # You can adjust the number of splits and other parameters

In [ ]:
# Iterate through models
for name, model in models:
    # Initialize lists to store scores for each fold
    ks_scores = []
    auc_scores = []
    prauc_scores = []  # Added for PRAUC
    f1_scores = []
    recall_scores = []
    precision_scores = []

    # Perform K-Fold cross-validation
    for train_index, val_index in kf.split(X_train_np):  # Use X_train_np
        X_train_fold, X_val_fold = X_train_np[train_index], X_train_np[val_index]
        y_train_fold, y_val_fold = y_train_np[train_index], y_train_np[val_index]  # Use y_train_np

        # Fit the model on the training fold
        model.fit(X_train_fold, y_train_fold)

        # Predict probabilities on the validation fold
        y_val_pred_prob = model.predict_proba(X_val_fold)[:, 1]

        # Calculate KS, AUC, PRAUC, F1-Score, Recall, and Precision for the fold
        ks_fold = custom_ks_scorer(y_val_fold, y_val_pred_prob)
        auc_fold = roc_auc_score(y_val_fold, y_val_pred_prob)
        prauc_fold = average_precision_score(y_val_fold, y_val_pred_prob)  # Calculate PRAUC
        f1_fold = f1_score(y_val_fold, (y_val_pred_prob > 0.5).astype(int))
        recall_fold = recall_score(y_val_fold, (y_val_pred_prob > 0.5).astype(int))
        precision_fold = precision_score(y_val_fold, (y_val_pred_prob > 0.5).astype(int))

        ks_scores.append(ks_fold)
        auc_scores.append(auc_fold)
        prauc_scores.append(prauc_fold)  # Store PRAUC
        f1_scores.append(f1_fold)
        recall_scores.append(recall_fold)
        precision_scores.append(precision_fold)

    # Store the mean scores in the results dictionary
    results_train_cv['Model'].append(name)
    results_train_cv['KS_Train_CV'].append(np.mean(ks_scores))
    results_train_cv['AUC_Train_CV'].append(np.mean(auc_scores))
    results_train_cv['PRAUC_Train_CV'].append(np.mean(prauc_scores))  # Store PRAUC
    results_train_cv['F1-Score_Train_CV'].append(np.mean(f1_scores))
    results_train_cv['Recall_Train_CV'].append(np.mean(recall_scores))
    results_train_cv['Precision_Train_CV'].append(np.mean(precision_scores))

In [ ]:
# Create a DataFrame for training set K-Fold cross-validation results
df_results_train_cv = pd.DataFrame(results_train_cv)

In [ ]:
# Display the DataFrame
print("K-Fold Cross-Validation Results on Training Set:")
print(df_results_train_cv)

df_results_train_cv.to_excel('/content/drive/MyDrive/GSS_Data/Fraud_train.xlsx')

In [ ]:
# Find the best model based on the highest mean KS score
best_model_ks = df_results_train_cv.loc[df_results_train_cv['KS_Train_CV'].idxmax()]['Model']

# Find the best model based on the highest mean AUC score
best_model_auc = df_results_train_cv.loc[df_results_train_cv['AUC_Train_CV'].idxmax()]['Model']

# Find the best model based on the highest mean F1-Score
best_model_f1 = df_results_train_cv.loc[df_results_train_cv['F1-Score_Train_CV'].idxmax()]['Model']

# Find the best model based on the highest mean Recall score
best_model_recall = df_results_train_cv.loc[df_results_train_cv['Recall_Train_CV'].idxmax()]['Model']

# Find the best model based on the highest mean Precision score
best_model_precision = df_results_train_cv.loc[df_results_train_cv['PRAUC_Train_CV'].idxmax()]['Model']

# Find the best model based on the highest mean Precision score
best_model_precision = df_results_train_cv.loc[df_results_train_cv['Precision_Train_CV'].idxmax()]['Model']

In [ ]:
# Display the best models for each metric
print("Best Models by Metric:")
print(f"KS: {best_model_ks}")
print(f"AUC: {best_model_auc}")
print(f"F1-Score: {best_model_f1}")
print(f"Recall: {best_model_recall}")
print(f"PRAUC: {best_model_precision}")
print(f"Precision: {best_model_precision}")

Generating KS, AUC, PRAUC, Precision, Recall and F1 for the test data

In [ ]:
#Predict the model on test data
model_predictions = {}
for name, model in models:
    predictions = model.predict(X_test)
    model_predictions[name] = predictions

In [ ]:
#Generating Performance Metrics for each model
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, precision_recall_curve, auc

model_metrics = {}

for name, predictions in model_predictions.items():
    # AUC
    auc_score = roc_auc_score(y_test, predictions)

    # F1-score
    f1 = f1_score(y_test, predictions)

    # Recall
    recall = recall_score(y_test, predictions)

    # Precision
    precision = precision_score(y_test, predictions)

    # Precision-Recall Curve and PRAUC
    precision_values, recall_values, _ = precision_recall_curve(y_test, predictions)
    pr_auc = auc(recall_values, precision_values)

    # KS Statistic (Custom Calculation)
    df_results = pd.DataFrame({'y_test': y_test, 'predictions': predictions})
    df_results['probability'] = model.predict_proba(X_test)[:, 1]
    grouped = df_results.groupby('probability')['y_test'].agg(['sum', 'count']).reset_index()
    grouped = grouped.rename(columns={'sum': 'events', 'count': 'non_events'})
    grouped['cum_events'] = grouped['events'].cumsum()
    grouped['cum_non_events'] = grouped['non_events'].cumsum()
    grouped['ks'] = abs(grouped['cum_events'] / sum(y_test) - grouped['cum_non_events'] / (len(y_test) - sum(y_test)))
    ks_statistic = grouped['ks'].max()

    model_metrics[name] = {
        'AUC': auc_score,
        'F1-Score': f1,
        'Recall': recall,
        'Precision': precision,
        'PRAUC': pr_auc,
        'KS': ks_statistic
    }

print("Model Performance Metrics:")
for name, metrics in model_metrics.items():
    print(f"Model: {name}")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value}")

In [ ]:
#Organising the performance into Dataframe
# Create a DataFrame from the model_metrics dictionary
results_df_test= pd.DataFrame(model_metrics).T

In [ ]:
# Rename the index column to 'Model'
results_df_test.index.name = 'Model'

In [ ]:
# Display the tabular results
print(results_df_test)

In [ ]:
#Exporting Data to excel
# Export the DataFrame to an Excel file
results_df_test.to_excel('/content/drive/MyDrive/GSS_Data/Fraud_test.xlsx')

In [ ]:
import shap

# Create a TreeExplainer from the Random Forest model
explainer = shap.Explainer(clf)
shap_values = explainer.shap_values(X_test)

# Plot SHAP summary plot
shap.summary_plot(shap_values, X_test, feature_names=feature_names, show=False)
plt.show()

In [ ]:
shap.summary_plot(shap_values, X_test, class_inds="original", class_names=model.classes_)

In [ ]:
shap.summary_plot(shap_values[1], X_test)

In [ ]:
!pip install initjs()

In [ ]:
 #Assuming you have shap_values calculated for all instances
# You can select shap_values for a specific instance by its index
instance_index = 10  # Replace with the index of the instance you want to plot

# Example shap_values array (replace this with your actual shap_values)
shap_values = explainer(X_test)  # Assuming you have shap values calculated

In [ ]:
#Water Fall plot

explainer =explainer  # Your SHAP explainer
x_test =X_test # Your test data

# Get SHAP values for the given test data
shap_values = explainer(x_test)

# Create an Explanation object using the SHAP values, base values, and feature names
exp = shap.Explanation(
    values=shap_values[:, :, 1],  # Replace [0] with the index of the output you want to explain
    base_values=shap_values.base_values[:, 1],  # Replace [0] with the index of the output you want to explain
    data=x_test,
    feature_names=x_test.columns
)

# Generate the waterfall plot for the selected output (index [0] in this example)
shap.waterfall_plot(exp[20])  # Replace [0] with the index of the output you want to explain

# Show the plot
plt.show()

In [ ]:
# Create the partial dependence plot
shap.plots.partial_dependence(
    'age', final_rf_model.predict, X_test,
    ice=False, model_expected_value=True, feature_expected_value=True
)

# Show the plot
plt.show()